In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, InputLayer, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import os

In [45]:
df = pd.read_csv('WeatherData/Quezon City Weather Data.csv')
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
features = ['temp', 'humidity', 'precip', 'windspeed']
df = df[features].copy().fillna(method='ffill')
df.tail()

C:\Users\User\AppData\Local\Temp\ipykernel_5404\3317473034.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df[features].copy().fillna(method='ffill')


,temp,humidity,precip,windspeed
datetime,,,,
2025-04-11,31.3,67.3,0.3,18.2
2025-04-12,31.5,63.7,0.1,16.3
2025-04-13,31.2,69.1,4.3,32.2
2025-04-14,31.2,68.5,0.7,15.3
2025-04-15,30.7,68.6,5.5,18.7


In [46]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

In [47]:
scaled_data

array([[0.28301887, 0.4766147 , 0.0013938 , 0.0690564 ],
       [0.29245283, 0.41202673, 0.        , 0.08592514],
       [0.33018868, 0.30734967, 0.        , 0.10595677],
       ...,
       [0.8490566 , 0.32293987, 0.00963557, 0.14496574],
       [0.8490566 , 0.30957684, 0.00156858, 0.0558777 ],
       [0.80188679, 0.31180401, 0.01232457, 0.07380074]])

In [48]:
def create_sequences(data, window_size=14, forecast_size=3):
    X, y = [], []
    for i in range(len(data) - window_size - forecast_size + 1):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size:i + window_size + forecast_size])
    return np.array(X), np.array(y)

In [49]:
X, y = create_sequences(scaled_data)
X = X.reshape((X.shape[0], X.shape[1], len(features)))
y = y.reshape(y.shape[0], -1)

In [50]:
model = Sequential([
    InputLayer((X.shape[1], X.shape[2])),
    LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.2),
    LSTM(64, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(4 * 3)  # 4 features * 3 days
])

In [51]:
optimizer = Adam(
    learning_rate=0.001,
    clipnorm=1.0  # Gradient clipping
)

In [52]:
model.compile(
    loss='mse',
    optimizer=optimizer,
    metrics=['mae']
)

In [53]:
os.makedirs("weatherModels", exist_ok=True)

In [54]:
checkPoint = ModelCheckpoint(
    filepath = 'weatherModels/best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode=min,
    verbose=1
)

C:\Users\User\AppData\Local\Temp\ipykernel_5404\2506179292.py:1: UserWarning: ModelCheckpoint mode '<built-in function min>' is unknown, fallback to auto mode.
  checkPoint = ModelCheckpoint(


In [55]:
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        min_delta=0.001,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    ),
    checkPoint
]

In [56]:
history = model.fit(
    X, y,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/100
145/147 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.1511 - mae: 0.2180
Epoch 1: val_loss improved from inf to 0.15880, saving model to weatherModels/best_model.keras
147/147 ━━━━━━━━━━━━━━━━━━━━ 13s 29ms/step - loss: 1.1401 - mae: 0.2162 - val_loss: 0.1588 - val_mae: 0.2048 - learning_rate: 0.0010
Epoch 2/100
144/147 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0559 - mae: 0.0727
Epoch 2: val_loss improved from 0.15880 to 0.05055, saving model to weatherModels/best_model.keras
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 0.0554 - mae: 0.0727 - val_loss: 0.0505 - val_mae: 0.1389 - learning_rate: 0.0010
Epoch 3/100
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0161 - mae: 0.0678
Epoch 3: val_loss improved from 0.05055 to 0.02714, saving model to weatherModels/best_model.keras
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 0.0161 - mae: 0.0678 - val_loss: 0.0271 - val_mae: 0.1030 - learning_rate: 0.0010
Epoch 4/100
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss:

In [37]:
best_model = load_model('weatherModels/best_model.keras')

In [38]:
def predict_future(model, data, scaler, window_size=14, forecast_days=3):
    last_window = scaler.transform(data[-window_size:])
    X_pred = last_window.reshape(1, window_size, len(features))
    pred = model.predict(X_pred)[0]
    pred = pred.reshape(forecast_days, len(features))
    return scaler.inverse_transform(pred)

In [39]:
next_3_days = predict_future(best_model, df.values, scaler)

c:\Users\User\Desktop\ForeLastDataTraining\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 649ms/step


In [40]:
forecast_dates = pd.date_range(
    start=df.index[-1] + pd.Timedelta(days=1),
    periods=3
)

In [41]:
output_df = pd.DataFrame(
    next_3_days,
    columns=features,
    index=forecast_dates
).reset_index()

In [42]:
output_df.insert(0, 'name', 'Quezon City, National Capital Region, Philippines')
output_df.rename(columns={'index': 'datetime'}, inplace=True)

In [43]:
final_output = output_df[['name', 'datetime'] + features]
print(final_output.to_string(index=False))

                                             name   datetime      temp  humidity    precip  windspeed
Quezon City, National Capital Region, Philippines 2025-04-16 30.720041 69.290123  5.017208  21.577856
Quezon City, National Capital Region, Philippines 2025-04-17 30.573256 70.099480  9.773944  21.044687
Quezon City, National Capital Region, Philippines 2025-04-18 30.543406 70.657524 10.415643  20.886862
